In [1]:
!pip install git+https://github.com/Navu45/image-merging.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/Navu45/image-merging.git to c:\users\alexey\appdata\local\temp\pip-req-build-7vivgtgt
  Resolved https://github.com/Navu45/image-merging.git to commit 7d37b93bbb006340273aa66ca8aee14626b56f4b
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/Navu45/image-merging.git 'C:\Users\Alexey\AppData\Local\Temp\pip-req-build-7vivgtgt'


In [2]:
import gradio as gr
import torch
from diffusers import DDIMScheduler
from mobile_sam import sam_model_registry, SamAutomaticMaskGenerator

from models.inpaint_image import InpaintPipeline, generate

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = InpaintPipeline.from_pretrained(
    'Fantasy-Studio/Paint-by-Example',
    torch_dtype=torch.float16,
    # local_files_only=True
)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_xformers_memory_efficient_attention()

model_type = "vit_t"
sam_checkpoint = "./models/weights/mobile_sam.pt"
mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam.to(device=device)
mobile_sam.eval()
mask_generator = SamAutomaticMaskGenerator(mobile_sam)

unet\diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You are using a model of type clip_vision_model to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [4]:
def save_images(*images):
    for i, image in enumerate(images):
        image.save(f'image{i}.png')

def create_img(source_image, target_image) -> list:
    source_image, target_image = source_image.resize((512, 512)), target_image.resize((512, 512))
    return generate(source_image, target_image, pipe, mask_generator)

with gr.Blocks() as demo:
    with gr.Tab(""):
        with gr.Row():
            source_image_input = gr.Image(type='pil')
            target_image_input = gr.Image(type='pil')
        with gr.Row():
            image_output = gr.Image()
        image_button = gr.Button("Generate image!")
    image_button.click(create_img, inputs=[source_image_input, target_image_input], outputs=image_output)

demo.launch(debug=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


  0%|          | 0/100 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.
